In [17]:
import os
import apbs
import numpy as np
import pdb2pqr
import subprocess
import re
from pathlib import Path

In [2]:
def run_pdb2pqr(pdb_path, pqr_path, forcefield='AMBER'):
    """
    Convert a PDB file to a PQR file using PDB2PQR.
    :param pdb_path: Path to the input PDB file.
    :param pqr_path: Path to the output PQR file.
    :param forcefield: Forcefield to use, default is 'parse'.
    """
    filename = os.path.splitext(os.path.basename(pdb_path))[0]
    pdb2pqr_cmd = [
        'pdb2pqr',
        '--ff={}'.format(forcefield),
        "--nodebump",
        "--noopt",
        "--apbs-input={}".format(filename + ".in"),
        pdb_path,
        pqr_path
    ]
    subprocess.run(pdb2pqr_cmd)

In [15]:
def run_apbs(apbs_input_path):
    apbs_input_path = os.path.abspath(apbs_input_path)

    print("Running APBS with:")
    print("Input file path:", apbs_input_path)

    apbs_cmd = [
        'apbs',
        apbs_input_path
    ]

    # Running the command and capturing output and errors
    process = subprocess.run(apbs_cmd, stdout=subprocess.PIPE, text=True)

    # Print the complete output (optional)
    print(process.stdout)

    # Parse output to find energy values
    energy_values = []
    for line in process.stdout.split('\n'):
        if "Local net energy (PE 0)" in line:
            # Extract the energy value using regex
            match = re.search(r"=\s*([\d.E+-]+)\s*kJ/mol", line)
            if match:
                energy_values.append(float(match.group(1)))
        elif "Global net ELEC energy" in line:
            match = re.search(r"=\s*([\d.E+-]+)\s*kJ/mol", line)
            if match:
                energy_values.append(float(match.group(1)))

    # Print the extracted values (optional)
    print('Local net energy', energy_values[0])
    print('Global net ELEC energy', energy_values[1])

    # You can now store these values in a file or return them
    return energy_values

In [9]:
pdb_path = 'ACE2.pdb'
pqr_path = 'ACE2.pqr'
run_pdb2pqr(pdb_path, pqr_path)

INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: ACE2.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 204 residues and 1559 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule.
INFO:Added atom OXT to residue GLY E 526 at coordinates -25.061, 14.727, 44.156
INFO:Updating disulfide bridges.
INFO:Adding hydrogens to biomolecule.
INFO:Optimizing hydrogen bonds
INFO:Initializing water bonding optimization...
INFO

In [16]:
apbs_input_path = 'ACE2.in'

energy_values = run_apbs(apbs_input_path)
# pdb2pqr --ff=AMBER --apbs-input=ACE2.in ACE2.pdb ACE2.pqr
# --noopt --nodebump
# python main.py /home/max/stayahead/snellius2/outputs/apbs/test/ACE2.in

Running APBS with:
Input file path: /home/max/stayahead/snellius2/scripts/analysis/apbs/ACE2.in


asc_getToken: Error occurred (bailing out).
Vio_scanf: Format problem with input.




----------------------------------------------------------------------
    APBS -- Adaptive Poisson-Boltzmann Solver
    Version 1.4.1
    
    Nathan A. Baker (nathan.baker@pnnl.gov)
    Pacific Northwest National Laboratory
    
    Additional contributing authors listed in the code documentation.
    
    Copyright (c) 2010-2012 Battelle Memorial Institute. Developed at the Pacific
    Northwest National Laboratory, operated by Battelle Memorial Institute, Pacific
    Northwest Division for the U.S. Department of Energy.
    
    Portions Copyright (c) 2002-2010, Washington University in St. Louis.
    Portions Copyright (c) 2002-2010, Nathan A. Baker.
    Portions Copyright (c) 1999-2002, The Regents of the University of California.
    Portions Copyright (c) 1995, Michael Holst.
    All rights reserved.
    
    Redistribution and use in source and binary forms, with or without
    modification, are permitted provided that the following conditions are met:
    
    * Redistribut

In [14]:
print(energy_values)

[166145.0639824, 166145.0639824]


In [19]:
def run_apbs_no(apbs_input_path):
    apbs_input_path = os.path.abspath(apbs_input_path)

    print("Running APBS with:")
    print("Input file path:", apbs_input_path)

    # # Running the command and capturing output and errors
    # process = subprocess.run(['apbs', apbs_input_path], shell=True, stdout=subprocess.PIPE)

    # print(process.stdout)

    apbs_cmd = [
        'apbs',
        apbs_input_path
    ]

    # Running the command and capturing output and errors
    process = subprocess.run(apbs_cmd)

In [ ]:
run_apbs_no(apbs_input_path)

In [31]:
def run_pdb2pqr(pdb_path, pqr_path, apbs_input_path, forcefield='AMBER'):
    """
    Convert a PDB file to a PQR file using PDB2PQR.
    :param pdb_path: Path to the input PDB file.
    :param pqr_path: Path to the output PQR file.
    :param forcefield: Forcefield to use, default is 'parse'.
    """
    pdb2pqr_cmd = [
        'pdb2pqr',
        '--ff={}'.format(forcefield),
        "--nodebump",
        "--noopt",
        "--apbs-input={}".format(apbs_input_path),
        pdb_path,
        pqr_path
    ]
    subprocess.run(pdb2pqr_cmd)

def run_apbs(apbs_input_path):
    directory, input_file = os.path.split(apbs_input_path)

    print("Running APBS with:")
    print("Input file path:", apbs_input_path)

    apbs_cmd = ['apbs', input_file]

    # Running the command and capturing output and errors
    process = subprocess.run(apbs_cmd, cwd=directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Print the complete output (optional)
    print(process.stdout)

    # Print any errors
    if process.stderr:
        print("Errors encountered:", process.stderr)

    # Parse output to find energy values
    energy_values = []
    for line in process.stdout.split('\n'):
        if "Local net energy (PE 0)" in line:
            # Extract the energy value using regex
            match = re.search(r"=\s*([\d.E+-]+)\s*kJ/mol", line)
            if match:
                energy_values.append(float(match.group(1)))
        elif "Global net ELEC energy" in line:
            match = re.search(r"=\s*([\d.E+-]+)\s*kJ/mol", line)
            if match:
                energy_values.append(float(match.group(1)))

    # Print the extracted values (optional)
    print('Local net energy', energy_values[0])
    print('Global net ELEC energy', energy_values[1])

    # You can now store these values in a file or return them
    return energy_values


def process_directory(input_dir,output_dir, type):
    results = []
    tmp_dir = os.path.join(output_dir, 'tmp')
    for subdir, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith("ranked_0.pdb") and type == 'alphafold':
                pdb_path = os.path.join(subdir, file)
                key_name = Path(subdir).stem
                print(key_name)
                pqr_dir = os.path.join(tmp_dir, key_name)
                os.makedirs(pqr_dir, exist_ok=True)
                pqr_path = os.path.abspath(f'{pqr_dir}/{key_name}.pqr')
                apbs_input_path = os.path.abspath(f'{pqr_dir}/{key_name}.in')
                run_pdb2pqr(pdb_path, pqr_path, apbs_input_path)
                if os.path.exists(apbs_input_path):
                    print(f"File found: {apbs_input_path}")
                else:
                    print(f"File not found: {apbs_input_path}")
                energy_values = run_apbs(apbs_input_path)
                results.append([key_name, energy_values[0], energy_values[1]])
            elif file.endswith(".pdb") and type == 'esmfold':
                pdb_path = os.path.join(subdir, file)
                key_name = file.split('.')[0]
                pqr_dir = os.path.join(tmp_dir, key_name)
                os.makedirs(os.path.join(pqr_dir, key_name), exist_ok=True)
                pdb2pqr_path = os.path.join(pqr_dir, key_name)
                run_pdb2pqr(pdb_path, pdb2pqr_path)
                apbs_input_path = os.path.join(pdb2pqr_path, key_name + '.in')
                energy_values = run_apbs(apbs_input_path)
                results.append([key_name, energy_values[0], energy_values[1]])
            print(results)

In [ ]:
input_dir = '/home/max/stayahead/out_tmp/alphafold/batch6_10'
output_dir = '/home/max/stayahead/snellius2/scripts/analysis/apbs/af_test'
type = 'alphafold'

process_directory(input_dir, output_dir, type)